# <font color=black> Figure 3 - Brain and spinal cord morphometry </font>
<hr style="border:1px solid black">

### Imports

In [ ]:
import sys,json, os
import pandas as pd
import numpy as np


main_dir="/cerebro/cerebro1/dataset/bmpd/derivatives/Aging_project/"
sys.path.append(main_dir + "2025_brsc_aging_project/code/")
from brain_post_cat12 import post_Cat12
from sc_structural_analyses import StructuralMetrics

#statistics
from brsc_statistics import Statistics

# plotting
from matplotlib.colors import LinearSegmentedColormap
from plotting import Plotting
import matplotlib.pyplot as plt

# 
#from scipy.stats import spearmanr
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Load config file ------------------------------------------------------------
config_file=main_dir + '/2025_brsc_aging_project/config/analyses/brsc_structural.json'
with open(config_file) as config_f: # the notebook should be in 'xx/notebook/' folder #config_proprio
    config = json.load(config_f) # load config file should be open first and the path inside modified



plot=Plotting(config_file,"test")
stat=Statistics(config=config,ana_dir="",analysis="")
post_cat12=post_Cat12(config=config)

<hr style="border:1px solid black">

## <font color=#0B7CC3> A. Brain volume (in ml)
#### Load cat12 files

In [ ]:
df_rois={}
for atlas_name in ["Schaefer2018_200Parcels_7Networks_order", "cobra"]:
    o_tag="_cobra" if atlas_name=="cobra" else "_Schaefer200"
    df_rois[atlas_name]=post_cat12.read_catROI(atlas_name=atlas_name,o_tag=o_tag,redo=False)
cobra_network=pd.read_csv(config["project_dir"] + "/templates/MNI/atlas/cobra/cobra.txt",delimiter="\t")
df_rois["cobra"]["networks"]=np.tile(cobra_network["networks"].values, len(config["participants_IDs_ALL"]))
brain_rois=pd.concat((df_rois["Schaefer2018_200Parcels_7Networks_order"],df_rois["cobra"]))
brain_grouped=brain_rois.groupby(["IDs","age","sex","networks"], sort=False)[["Vgm","Vwm"]].sum().reset_index()
brain_grouped["value"]=brain_grouped["Vgm"]

### Load spinal cord results

In [ ]:
config["participants_IDs_T2s"]=config["participants_IDs_ALL"]
sc_morpho=StructuralMetrics(config,contrast="T2s") # initialize the function
files,df_csa=sc_morpho.compute_csa(i_tag="gm",n_jobs=1,redo_indiv=False)
df_csa_grouped=df_csa[(df_csa["level_labels"]!="C1") & (df_csa["level_labels"]!="C8")].groupby(["IDs","age","sex","groups"], sort=False)[["MEAN(area)"]].mean().reset_index()
df_csa_grouped["networks"]="SpinalCord"
df_csa_grouped["value"]=df_csa_grouped["MEAN(area)"]

df_morpho_all=pd.concat((brain_grouped,df_csa_grouped),axis=0)

#### Plot brain results (SomMot)

In [ ]:
from statsmodels.stats.multitest import multipletests
output_dir_table=config["project_dir"] +  "/figures/f01_structural/revision_R1/brain/"

from statsmodels.stats.multitest import fdrcorrection
colors = ["#0C675F","#3AA198","#68BCB4","#86C9C3","#C3E4E1","#F1C2CF","#E3849F","#D5476F","#C7093F","#850429"]  # blue → white → red
custom_cmap = LinearSegmentedColormap.from_list("my_colormap", colors)
output_dir=config["project_dir"] +config["analysis_dir"]['cat12'].split("brain")[0] + "/figures/"


dfs = []  # should be a DataFrame, not dict
network_df = {}  # this is fine as dict

for network in ['Vis', 'SomMot', 'DorsAttn', 'SalVentAttn', 'Limbic', 'Cont','Default', 'Subcortical', 'Cerebellum',"SpinalCord"]:
    print(f"Processing network: {network}")
    roi_subresults = [];
    network_df[network] = df_morpho_all[df_morpho_all["networks"] == network].reset_index(drop=True)
    
    if not os.path.exists(output_dir_table + "stats_"+network+"_vol_csa.csv"):
        signed_r2, p_age,p_sex, beta_age,beta_sex, stat_age,stat_sex,ci_r2_age,  ci_beta_age, ci_beta_sex =stat.regression_model(df=network_df[network], y="value", predictor="age", covariates=["sex"], n_bootstrap=1000)
        roi_subresults.append({"Network": network,"Predictor": "age","β": beta_age,"95% Bootstrap CI": ci_beta_age,"t-value": stat_age,"p-value": p_age,"p-fdr": p_age,})
        roi_subresults.append({"Network": network,"Predictor": "sex","β": beta_sex,"95% Bootstrap CI": ci_beta_sex,"t-value": stat_sex,"p-value": p_sex,"p-fdr": p_sex,})
        roi_subresults_df= pd.DataFrame(roi_subresults)
        roi_subresults_df.to_csv(output_dir_table + "stats_"+network+"_vol_csa.csv", index=False)
            
    else:
        roi_subresults_df = pd.read_csv(output_dir_table + "stats_"+network+"_vol_csa.csv")

    p_val=int(roi_subresults_df[roi_subresults_df["Predictor"]=="age"]["p-value"][0])
    t_val=int(roi_subresults_df[roi_subresults_df["Predictor"]=="age"]["t-value"][0])
    print(f"t-value (age): {t_val:.2f}, p-value (age): {p_val:.8f}")
    plot.lmplots(
        df=network_df[network],
        x_data="age",y_data="value",
        color=["grey"],
        hue_color_var="age",hue_palette=custom_cmap,
        xmin=15,xmax=90,
        output_dir=output_dir,
        output_tag="aging_" + network + "_vol",
        indiv_values=True,save=False)

    dfs.append(roi_subresults_df)
df_all = pd.concat(dfs, ignore_index=True)#pd.concat([df_all, roi_subresults_df], ignore_index=True)


# FDR correction
df_all["p-fdr"] = None  # initialize column

if not os.path.exists(output_dir_table + "stats_all_vol_csa.csv"):
    for predictor in df_all["Predictor"].unique():
        mask = df_all["Predictor"] == predictor
        pvals = df_all.loc[mask, "p-value"].values
    
        _, pvals_fdr, _, _ = multipletests(pvals,alpha=0.05,method="fdr_bh")
    
        df_all.loc[mask, "p-fdr"] = pvals_fdr
    df_all.to_csv(output_dir_table + "stats_all_vol_csa.csv", index=False)
else:
    df_all = pd.read_csv(output_dir_table + "stats_all_vol_csa.csv")

#all_networks_df = pd.concat(network_df.values(), ignore_index=True)
#all_networks_df.to_csv(config["project_dir"] +  "/figures/source_datafile/fig_4A_S7_brsc_morpho.csv", index=False)

In [ ]:
#all_networks_df = pd.concat(network_df.values(), ignore_index=True)
#all_networks_df.to_csv(config["project_dir"] +  "/figures/source_datafile/fig_4A_S7_brsc_morpho.csv", index=False)

<hr style="border:1px solid black">

## <font color=#0B7CC3> B.  Link brain / spinal cord


In [ ]:
df=df_all[df_all["Predictor"]=="age"]
color=["#452B7A","#452B7A","#452B7A","#452B7A"]
plot.barplots(df=df, x_data="Network",x_order=["SpinalCord","Subcortical","Cerebellum","SomMot"], #x_order=["ventral","dorsal","within","cross"],
                  y_data="t-value",
                  palette=color,
                  ymin=-7,ymax=0,indiv_values=False,
                output_dir=output_dir ,
                  #height=3,aspect=0.5,
                  output_tag="age_effect_tvalues_brsc_GM",save=False)
plt.show()
#df.to_csv(config["project_dir"] +  "/figures/source_datafile/fig_4B_brsc_morpho_aging.csv", index=False)

In [ ]:
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm

colors = ["#0C675F","#3AA198","#68BCB4","#86C9C3","#C3E4E1","#F1C2CF","#E3849F","#D5476F","#C7093F","#850429"]  # blue → white → red
custom_cmap = LinearSegmentedColormap.from_list("my_colormap", colors)

networks_to_keep = ["SomMot", "Subcortical", "Cerebellum", "SpinalCord"]  # customize as needed
df_filtered = df_morpho_all[df_morpho_all['networks'].isin(networks_to_keep)]
df_pivot = df_filtered.pivot_table(index=['IDs', 'age', 'sex'],columns='networks',values='value').reset_index()



#---- Plot resuts and compute stats
limits={"SomMot":[35,75], "Subcortical":[15,35], "Cerebellum":[65,105]}
for network_nb,network in enumerate(networks_to_keep[:-1]):
    model = smf.ols('SpinalCord ~ ' + network+' * age + sex', data=df_pivot).fit()
    print(model.summary())



    plot.lmplots(df=df_pivot,
                 x_data=network,
                 y_data="SpinalCord", 
                 color=["grey"],hue_color_var="age",hue_palette=custom_cmap,
                 height=2.5,aspect=1.5,
                 ymin=10,ymax=20,
                 xmin=limits[network][0],xmax=limits[network][1],
             indiv_values=True,
                  output_dir=output_dir,output_tag="csa_" + network + "_vol",
                 save=True)
    
    #print(model.summary()) 
